In [3]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd

In [4]:
#scrape single game shots
base_url = 'https://understat.com/match/'
match = str(input('Please enter the match id'))
url = base_url+match

Please enter the match id21928


In [5]:
#use requests and beautifulsoup to parse site
res = requests.get(url)
soup = BeautifulSoup(res.content,'lxml')
scripts = soup.find_all('script')

In [6]:
#filter for shot data
strings = scripts[1].string

In [7]:
strings

"\n\tvar shotsData \t= JSON.parse('\\x7B\\x22h\\x22\\x3A\\x5B\\x7B\\x22id\\x22\\x3A\\x22536219\\x22,\\x22minute\\x22\\x3A\\x226\\x22,\\x22result\\x22\\x3A\\x22Goal\\x22,\\x22X\\x22\\x3A\\x220.9430000305175781\\x22,\\x22Y\\x22\\x3A\\x220.8269999694824218\\x22,\\x22xG\\x22\\x3A\\x220.06246471777558327\\x22,\\x22player\\x22\\x3A\\x22Mathias\\x20Jensen\\x22,\\x22h_a\\x22\\x3A\\x22h\\x22,\\x22player_id\\x22\\x3A\\x227166\\x22,\\x22situation\\x22\\x3A\\x22DirectFreekick\\x22,\\x22season\\x22\\x3A\\x222023\\x22,\\x22shotType\\x22\\x3A\\x22RightFoot\\x22,\\x22match_id\\x22\\x3A\\x2221928\\x22,\\x22h_team\\x22\\x3A\\x22Brentford\\x22,\\x22a_team\\x22\\x3A\\x22Bournemouth\\x22,\\x22h_goals\\x22\\x3A\\x222\\x22,\\x22a_goals\\x22\\x3A\\x222\\x22,\\x22date\\x22\\x3A\\x222023\\x2D09\\x2D02\\x2014\\x3A00\\x3A00\\x22,\\x22player_assisted\\x22\\x3Anull,\\x22lastAction\\x22\\x3A\\x22Standard\\x22\\x7D,\\x7B\\x22id\\x22\\x3A\\x22536220\\x22,\\x22minute\\x22\\x3A\\x2210\\x22,\\x22result\\x22\\x3A\\x22Bloc

In [11]:
#only filter for Json data
ind_start = strings.index("('")+2
ind_end = strings.index("')")
json_data = strings[ind_start:ind_end]
json_data = json_data.encode('utf8').decode('unicode_escape')

#convert strings to json format
data = json.loads(json_data)
                        

In [12]:
data

{'h': [{'id': '536219',
   'minute': '6',
   'result': 'Goal',
   'X': '0.9430000305175781',
   'Y': '0.8269999694824218',
   'xG': '0.06246471777558327',
   'player': 'Mathias Jensen',
   'h_a': 'h',
   'player_id': '7166',
   'situation': 'DirectFreekick',
   'season': '2023',
   'shotType': 'RightFoot',
   'match_id': '21928',
   'h_team': 'Brentford',
   'a_team': 'Bournemouth',
   'h_goals': '2',
   'a_goals': '2',
   'date': '2023-09-02 14:00:00',
   'player_assisted': None,
   'lastAction': 'Standard'},
  {'id': '536220',
   'minute': '10',
   'result': 'BlockedShot',
   'X': '0.805',
   'Y': '0.4809999847412109',
   'xG': '0.06323494017124176',
   'player': 'Vitaly Janelt',
   'h_a': 'h',
   'player_id': '9680',
   'situation': 'OpenPlay',
   'season': '2023',
   'shotType': 'LeftFoot',
   'match_id': '21928',
   'h_team': 'Brentford',
   'a_team': 'Bournemouth',
   'h_goals': '2',
   'a_goals': '2',
   'date': '2023-09-02 14:00:00',
   'player_assisted': None,
   'lastAction':

In [19]:
x = []
y = []
xG = []
result = []
team = []
data_away = data['a']
data_home = data['h']

for index in range(len(data_home)):
    for key in data_home[index]:
        if key == 'X':
            x.append(data_home[index][key])
        if key == 'Y':
            y.append(data_home[index][key])
        if key == 'h_team':
            team.append(data_home[index][key])
        if key == 'xG':
            xG.append(data_home[index][key])
        if key == 'result':
            result.append(data_home[index][key])
            
for index in range(len(data_away)):
    for key in data_away[index]:
        if key == 'X':
            x.append(data_away[index][key])
        if key == 'Y':
            y.append(data_away[index][key])
        if key == 'a_team':
            team.append(data_away[index][key])
        if key == 'xG':
            xG.append(data_away[index][key])
        if key == 'result':
            result.append(data_away[index][key])

In [22]:
col_names = ['x','y','xG','result','team']
df = pd.DataFrame([x,y,xG,result,team],index=col_names)
df = df.T

In [23]:
df

,x,y,xG,result,team
0,0.9430000305175781,0.8269999694824218,0.06246471777558327,Goal,Brentford
1,0.805,0.4809999847412109,0.06323494017124176,BlockedShot,Brentford
2,0.84,0.47700000762939454,0.3208167552947998,SavedShot,Brentford
3,0.8730000305175781,0.36400001525878906,0.09073782712221146,MissedShots,Brentford
4,0.9340000152587891,0.535,0.41993844509124756,MissedShots,Brentford
5,0.7419999694824219,0.5059999847412109,0.019839800894260406,MissedShots,Brentford
6,0.9109999847412109,0.4490000152587891,0.44571805000305176,MissedShots,Brentford
7,0.9230000305175782,0.46900001525878904,0.1277308315038681,SavedShot,Brentford
8,0.9230000305175782,0.4909999847412109,0.06615801155567169,MissedShots,Brentford
9,0.9280000305175782,0.5740000152587891,0.026131179183721542,MissedShots,Brentford


In [25]:
file_path = "/Users/calvi/Downloads/Brentford Vs Bournemouth Sep 02.csv"
df.to_csv(file_path, index=False)